In [1]:
!ls /usr/local/cuda*

/usr/local/cuda:
bin		   EULA.txt  libnvvp	       nvvm-prev  src
compat		   extras    nsightee_plugins  README	  targets
compute-sanitizer  include   nvml	       samples	  tools
DOCS		   lib64     nvvm	       share	  version.json

/usr/local/cuda-11:
bin		   EULA.txt  libnvvp	       nvvm-prev  src
compat		   extras    nsightee_plugins  README	  targets
compute-sanitizer  include   nvml	       samples	  tools
DOCS		   lib64     nvvm	       share	  version.json

/usr/local/cuda-11.2:
bin		   EULA.txt  libnvvp	       nvvm-prev  src
compat		   extras    nsightee_plugins  README	  targets
compute-sanitizer  include   nvml	       samples	  tools
DOCS		   lib64     nvvm	       share	  version.json


In [2]:
!nvidia-smi

Tue Nov 29 08:17:42 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!git clone https://github.com/rmaksymiuk/rodinia-gpu

Cloning into 'rodinia-gpu'...
remote: Enumerating objects: 2140, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 2140 (delta 16), reused 75 (delta 14), pack-reused 2063
Receiving objects: 100% (2140/2140), 64.00 MiB | 13.86 MiB/s, done.
Resolving deltas: 100% (1007/1007), done.
Checking out files: 100% (1394/1394), done.


Application: k-Nearest 
Neighbors Dwarf: Dense Linear Algebra

In [5]:
%cd /content/rodinia-gpu/cuda/nn
!make nn
!./nn filelist_4 -r 3 -lat 30 -lng 90

/content/rodinia-gpu/cuda/nn
make: 'nn' is up to date.
1988 12 27  0 18 TONY       30.0  89.8  113   39 --> Distance=0.199997
1980 10 22 18  3 ISAAC      30.1  90.4  110  778 --> Distance=0.412312
1997 11 14 12 24 HELENE     30.5  89.8  134  529 --> Distance=0.538515


In [6]:
!nvprof --print-gpu-trace ./nn filelist_4 -r 3 -lat 30 -lng 90

==416== NVPROF is profiling process 416, command: ./nn filelist_4 -r 3 -lat 30 -lng 90
1988 12 27  0 18 TONY       30.0  89.8  113   39 --> Distance=0.199997
1980 10 22 18  3 ISAAC      30.1  90.4  110  778 --> Distance=0.412312
1997 11 14 12 24 HELENE     30.5  89.8  134  529 --> Distance=0.538515
==416== Profiling application: ./nn filelist_4 -r 3 -lat 30 -lng 90
==416== Profiling result:
   Start  Duration            Grid Size      Block Size     Regs*    SSMem*    DSMem*      Size  Throughput  SrcMemType  DstMemType           Device   Context    Stream  Name
484.37ms  31.135us                    -               -         -         -         -  334.09KB  10.233GB/s    Pageable      Device     Tesla T4 (0)         1         7  [CUDA memcpy HtoD]
484.41ms  6.4320us            (168 1 1)       (256 1 1)        16        0B        0B         -           -           -           -     Tesla T4 (0)         1         7  euclid(latLong*, float*, int, float, float) [118]
484.44ms  15.168us    

In [7]:
%cd /content/rodinia-gpu/openmp/nn
!make nn
!./nn filelist_4 3 30 90

/content/rodinia-gpu/openmp/nn
make: 'nn' is up to date.
The 3 nearest neighbors are:
1980 10 22 18  3 ISAAC      30.1  90.4  110  778 --> 0.412312
1997 11 14 12 24 HELENE     30.5  89.8  134  529 --> 0.538515
1988 12 27  0 18 TONY       30.0  89.8  113   39 --> 0.199997
total time :  0.042224001139 s

In [29]:
%cd /content/rodinia-gpu/cuda/bfs
!make
!nvprof --profile-child-processes ./run

/content/rodinia-gpu/cuda/bfs
/usr/local/cuda/bin/nvcc bfs.cu -o bfs -I/usr/local/cuda/include -L/usr/local/cuda/lib64 
Reading File
Read File
==1562== NVPROF is profiling process 1562, command: ./bfs ../../data/bfs/graph1MW_6.txt
Copied Everything to GPU memory
Start traversing the tree
Kernel Executed 12 times
Result stored in result.txt
==1562== Profiling application: ./bfs ../../data/bfs/graph1MW_6.txt
==1562== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   64.75%  7.5863ms        18  421.46us  1.3760us  5.0135ms  [CUDA memcpy HtoD]
                   26.28%  3.0793ms        12  256.61us  36.384us  1.1349ms  Kernel(Node*, int*, bool*, bool*, bool*, int*, int)
                    5.02%  588.44us        13  45.264us  1.6640us  567.25us  [CUDA memcpy DtoH]
                    3.94%  461.43us        12  38.452us  31.743us  46.463us  Kernel2(bool*, bool*, bool*, bool*, int)
      API calls:   95.26%  315.28ms       

In [32]:
%cd /content/rodinia-gpu/openmp/bfs
!make
!./run

/content/rodinia-gpu/openmp/bfs
icc -g -fopenmp -O2 -DOMP_OFFLOAD bfs.cpp -o bfs_offload
make: icc: Command not found
Makefile:12: recipe for target 'bfs_offload' failed
make: *** [bfs_offload] Error 127
Reading File
Start traversing the tree
Compute time: 0.064479
Result stored in result.txt


Application: PathFinder
Dwarf: Dynamic Programming

In [36]:
%cd /content/rodinia-gpu/cuda/pathfinder/
!make
!./run
!nvprof --profile-child-processes  ./run

/content/rodinia-gpu/cuda/pathfinder
/usr/local/cuda/bin/nvcc pathfinder.cu -o pathfinder -I/usr/local/cuda/include -L/usr/local/cuda/lib64 
==1852== NVPROF is profiling process 1852, command: ./pathfinder 100000 100 20
==1852== Profiling application: ./pathfinder 100000 100 20
==1852== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   95.08%  8.2998ms         2  4.1499ms  35.935us  8.2638ms  [CUDA memcpy HtoD]
                    4.54%  396.73us         5  79.345us  77.054us  80.542us  dynproc_kernel(int, int*, int*, int*, int, int, int, int)
                    0.38%  32.928us         1  32.928us  32.928us  32.928us  [CUDA memcpy DtoH]
      API calls:   96.49%  305.95ms         3  101.98ms  4.6800us  305.78ms  cudaMalloc
                    2.93%  9.3028ms         3  3.1009ms  106.44us  8.4554ms  cudaMemcpy
                    0.40%  1.2529ms         3  417.63us  6.8820us  1.0748ms  cudaFree
                    0.1

In [39]:
%cd /content/rodinia-gpu/openmp/pathfinder/
!make
%time !./run

/content/rodinia-gpu/openmp/pathfinder
g++ pathfinder.cpp -fopenmp -o pathfinder
CPU times: user 7.72 ms, sys: 8.35 ms, total: 16.1 ms
Wall time: 920 ms
